# Silver Layer

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

##DATA ACCESS

In [0]:
spark.conf.set("fs.azure.account.auth.type.storageeeaccount.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.storageeeaccount.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.storageeeaccount.dfs.core.windows.net", "<AZURE_CLIENT_ID_FROM_KEY_VAULT>")
spark.conf.set("fs.azure.account.oauth2.client.secret.storageeeaccount.dfs.core.windows.net", dbutils.secrets.get(scope="my-scope", key="client-secret"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.storageeeaccount.dfs.core.windows.net", "https://login.microsoftonline.com/<TENANT_ID>/oauth2/token")


##DATA LOADING

Read Data

In [0]:
df_cal=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/adventure_calender')

In [0]:
df_pro_cat=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/adventure_product_categories')

In [0]:
df_work_cust=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/adventureworks_customers')

In [0]:
df_prod=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/product')

In [0]:
df_prod_sub=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/product_subcatagories')

In [0]:
df_return=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/returns')

In [0]:
df_sales=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/sales_*')

In [0]:
df_ter=spark.read.format('csv')\
    .option("header",True).\
        option("inferSchema",True)\
            .load('abfss://bronze@storageeeaccount.dfs.core.windows.net/territories')

###Transformation

In [0]:
df_cal.display()

In [0]:
df_cal=df_cal.withColumn('Month',month(col('Date')))\
           .withColumn('year',year(col('Date')))

df_cal.display()

In [0]:
df_cal.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/adventure_calender").save()

In [0]:
df_work_cust.display()

In [0]:
df_work_cust=df_work_cust.withColumn("Full Name",concat_ws(" ",col("Prefix"),col("FirstName"),col("LastName")))

In [0]:
df_work_cust.display()

In [0]:
df_work_cust.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/adventure_works_customer").save()

###sub categories

In [0]:
df_prod_sub.display()

ProductSubcategoryKey,SubcategoryName,ProductCategoryKey
1,Mountain Bikes,1
2,Road Bikes,1
3,Touring Bikes,1
4,Handlebars,2
5,Bottom Brackets,2
6,Brakes,2
7,Chains,2
8,Cranksets,2
9,Derailleurs,2
10,Forks,2


In [0]:
df_prod_sub.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/product_subcatagories").save()

###Products

In [0]:
df_prod.display()

In [0]:
df_prod=df_prod.withColumn("ProductSKU",split(col("ProductSKU"),'-')[0])\
    .withColumn("ProductName",split(col("ProductName"),' ')[0])


In [0]:
df_prod.display()

In [0]:
df_prod.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/product").save()


###Returns

In [0]:
df_return.display()

In [0]:
df_return.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/returns").save()

###Territories

In [0]:
df_ter.display()

SalesTerritoryKey,Region,Country,Continent
1,Northwest,United States,North America
2,Northeast,United States,North America
3,Central,United States,North America
4,Southwest,United States,North America
5,Southeast,United States,North America
6,Canada,Canada,North America
7,France,France,Europe
8,Germany,Germany,Europe
9,Australia,Australia,Pacific
10,United Kingdom,United Kingdom,Europe


In [0]:
df_ter.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/territories").save()


###Sales

In [0]:
df_sales.display()

In [0]:
df_sales=df_sales.withColumn("StockDate",to_timestamp(col("StockDate")))

In [0]:
df_sales=df_sales.withColumn('OrderNUmber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sales=df_sales.withColumn("MULTIPLY",col("OrderLineItem")*col("OrderQuantity"))

In [0]:
df_sales.display()

In [0]:
df_sales.groupBy("OrderDate").agg(count("OrderNUmber").alias("Total Orders")).display()

In [0]:
df_sales.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/sales").save()

In [0]:
df_prod.display()

In [0]:
df_pro_cat.display()

ProductCategoryKey,CategoryName
1,Bikes
2,Components
3,Clothing
4,Accessories


Databricks visualization. Run in Databricks to view.

In [0]:
df_pro_cat.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/adventure_product_categories").save()

In [0]:
df_prod.groupBy("ProductSubcategoryKey").agg(count("ProductName").alias("Total Products")).orderBy("ProductSubcategoryKey").display()


ProductSubcategoryKey,Total Products
1,32
2,43
3,22
4,8
5,3
6,2
7,1
8,3
9,2
10,3


Databricks visualization. Run in Databricks to view.

In [0]:
df_prod.write.format('parquet').mode('append').option("path","abfss://silver@storageeeaccount.dfs.core.windows.net/products").save

In [0]:
df_ter.display()

SalesTerritoryKey,Region,Country,Continent
1,Northwest,United States,North America
2,Northeast,United States,North America
3,Central,United States,North America
4,Southwest,United States,North America
5,Southeast,United States,North America
6,Canada,Canada,North America
7,France,France,Europe
8,Germany,Germany,Europe
9,Australia,Australia,Pacific
10,United Kingdom,United Kingdom,Europe


Databricks visualization. Run in Databricks to view.